In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#df_sim.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats//df_all_round_sim.csv', index=None)
#df_sim['season'] = 1

df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats//df_all_round_sim.csv')

df_sim2 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

df_sim = pd.concat([df_sim,df_sim2], axis=0).reset_index(drop=True)

matches_list1 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list2 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list1['match_id'] = matches_list1['num'].apply(lambda x: 'M00' + str(x) if x<=9 else 'M0' + str(x))
matches_list2['match_id'] = matches_list2['num'].apply(lambda x: 'S02M00' + str(x) if x<=9 else 'S02M0' + str(x))

matches_list = pd.concat([matches_list1[['num','Spin','match_id']],matches_list2[['num','Spin','match_id']]],
                         axis=0)
df_sim = df_sim.merge(matches_list[['match_id','Spin']], on='match_id', how='left')
df_sim['pitch'] = df_sim['Spin'].apply(lambda x: 'pace' if x<=40 
                                       else 'spin' if x>=60
                                      else 'neutralish')

df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_03/Stats/df_combined_SPL.csv')

for column in df_sim.columns:
    df_sim[column] = df_sim[column].replace('Robin Minz','R Minz')
    
df_sim.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_03/Stats/df_combined_SPL.csv', index=None)

In [23]:
df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_03/Stats/df_combined_SPL.csv')

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'wicket_type', 'player_dismissed', 'legal_balls_bowled', 'wicket_event',
       'fielder', 'legality', 'bowler_wicket', 'islegal', 'isDotforBatter',
       'isOne', 'isTwo', 'isThree', 'isStrikeRotation', 'isFour', 'isSix',
       'isBowlerWicket', 'is_striker_Out', 'is_nonstriker_Out', 'isWicket',
       'total_runs', 'runs_scored', 'wickets_down', 'run_rate', 'last_fow',
       'partnership_runs', 'isno', 'is_faced_by_batter', 'last_fow_run_rate',
       'fours', 'sixes', 'striker_batting_position',
       'non_striker_batting_position', 'runs_conceeded', 'isDotforbowler',
       'legal_balls_remaining', 'target', 'runs_remaining', 'reqd_run_rate',
       'over', 'Spin', 'pitch'],
      dtype='object')

###### updation of player names

In [34]:
updates_file = pd.read_excel("/Users/roumyadas/Desktop/IPL_Simulation/Season_03/updates_/separate_stats.xlsx",
                             sheet_name=None)

sheet_names = list(updates_file.keys())
all_sheets_list = list(updates_file.values())

for df in all_sheets_list:
    for column in df.columns:
        df[column] = df[column].replace('Vaibhav Suryavanshi', 'V Suryavanshi')
        df[column] = df[column].replace('Ayush Mhatre', 'A Mhatre')
        df[column] = df[column].replace('Robin Minz', 'R Minz')

with pd.ExcelWriter("/Users/roumyadas/Desktop/IPL_Simulation/Season_03/updates_/separate_stats.xlsx", 
                    engine='openpyxl', mode='w') as writer:
    for sheet_name, df in zip(sheet_names, all_sheets_list):
        df.to_excel(writer, sheet_name=sheet_name, index=False)


In [19]:
df = pd.DataFrame(columns=['match_id','venue',
                           '1st_innings_total','1st_innings_wkts',
                           '1st_bat_team','2nd_bat_team',
                           '2nd_innings_total','2nd_innings_wkts',
                           'pitch','bat_1st_win','bat_2nd_win','winner'])


for m in df_sim.match_id.unique()[:-1]:
    sub_df = df_sim[df_sim.match_id == m]

    bat1 = sub_df[sub_df.innings==1]['runs_scored'].max()
    wkt1 = sub_df[sub_df.innings==1]['wickets_down'].max()
    bat2 = sub_df[sub_df.innings==2]['runs_scored'].max()
    wkt2 = sub_df[sub_df.innings==2]['wickets_down'].max()
    venue = sub_df['venue'].unique()[0]
    pitch = sub_df['pitch'].unique()[0]
    t1 = sub_df[sub_df.innings==1]['batting_team'].unique()[0]
    t2 = sub_df[sub_df.innings==2]['batting_team'].unique()[0]
    winner = (t1 if bat1>bat2 else t2)
    bat_1st_win = (1 if bat1>bat2 else 0)
    bat_2nd_win = (1 if bat1<bat2 else 0)

    row_ = [m,venue,bat1,wkt1,t1,t2,bat2,wkt2,pitch,bat_1st_win,bat_2nd_win,winner]
    row_df = pd.DataFrame([row_], columns=df.columns)
    df = pd.concat([df, row_df], ignore_index=True)

    


In [21]:
df.to_excel('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/venue_stats_indi.xlsx', index=None)

##### example calc